# setup

In [2]:
!pip install datasets tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

# data

In [5]:
from datasets import load_dataset
dataset = load_dataset('EleutherAI/pile', split='train', streaming=True)

In [15]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function)


In [16]:
import torch
from torch.utils.data import DataLoader, IterableDataset

class StreamingDataset(IterableDataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __iter__(self):
        for item in self.dataset:
            yield item

streaming_dataset = StreamingDataset(tokenized_datasets)
train_dataloader = DataLoader(streaming_dataset, batch_size=8)


# model

In [17]:
import torch
from torch import nn

class SimpleTransformer(nn.Module):
    def __init__(self, d_model, d_ff, vocab_size):
        super(SimpleTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.transformer_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=8, dim_feedforward=d_ff, activation='relu')
        self.transformer = nn.TransformerEncoder(self.transformer_layer, num_layers=1)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        x = self.transformer(x)
        x = self.fc(x)
        return x

# Model parameters
d_model = 128
d_ff = 512
vocab_size = 50257  # Size of tokenizer vocabulary (e.g., GPT-2 tokenizer size)

model = SimpleTransformer(d_model=d_model, d_ff=d_ff, vocab_size=vocab_size).cuda()


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
!pip install zstandard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 16.8 MB/s eta 0:00:00


In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm

# Training parameters
num_epochs = 1
learning_rate = 5e-5
max_steps = 10000  # Set a fixed number of training steps

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=max_steps)

# Training loop
model.train()
step = 0
for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader):
        if step >= max_steps:
            break

        input_ids = batch['input_ids'].cuda()
        labels = batch['input_ids'].cuda()  # Use the input as labels for language modeling

        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = nn.CrossEntropyLoss()(outputs.view(-1, vocab_size), labels.view(-1))
        loss.backward()
        optimizer.step()
        scheduler.step()

        step += 1

    if step >= max_steps:
        break

    print(f"Epoch {epoch+1}/{num_epochs} completed. Loss: {loss.item()}")


0it [00:00, ?it/s]


FileNotFoundError: https://the-eye.eu/public/AI/pile/train/00.jsonl.zst